In [1]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    FACTOR_MODEL_ESTIMATES
)
from typing import Dict


In [3]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "database_server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django

django.setup()

symbol = "BTC-USDfewfew"



In [4]:
from crypto_calculator.models import RawPriceData


raw_price_data = RawPriceData.objects.filter(symbol=symbol).values("close", "date")
df = pd.DataFrame(list(raw_price_data))
if len(list(raw_price_data)) > 0:
    df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df.set_index("date", inplace=True, drop=True)

In [7]:
len(df)

0

In [5]:
def create_ewma_std_estimates(df: pd.DataFrame, halflife: float=365, min_periods: float=20) -> pd.DataFrame:
    return df.ewm(halflife=halflife, min_periods=min_periods).std().dropna()

create_ewma_std_estimates(df).to_dict()

{'return_field': {'2014-10-08': 0.041320419758777996,
  '2014-10-09': 0.04133230005676715,
  '2014-10-10': 0.04032288239973414,
  '2014-10-11': 0.03941722628008253,
  '2014-10-12': 0.039936765955637214,
  '2014-10-13': 0.03972070570089154,
  '2014-10-14': 0.03932888351500317,
  '2014-10-15': 0.03864195800977953,
  '2014-10-16': 0.0383091278454704,
  '2014-10-17': 0.03761786151372875,
  '2014-10-18': 0.03718359354789838,
  '2014-10-19': 0.03654505867628458,
  '2014-10-20': 0.036036958969801754,
  '2014-10-21': 0.03551529034449181,
  '2014-10-22': 0.03497446643503595,
  '2014-10-23': 0.03606097346960777,
  '2014-10-24': 0.035531285357402825,
  '2014-10-25': 0.03530449425977824,
  '2014-10-26': 0.03507444075960042,
  '2014-10-27': 0.03459346472578174,
  '2014-10-28': 0.034242750921131805,
  '2014-10-29': 0.03503067806980499,
  '2014-10-30': 0.03499620913435391,
  '2014-10-31': 0.03464945013515633,
  '2014-11-01': 0.03459087511892183,
  '2014-11-02': 0.03419069505197273,
  '2014-11-03': 0.

In [8]:
def create_ewma_std_estimates(df: pd.DataFrame, halflife: float=365, min_periods: float=20) -> pd.DataFrame:
    return df.ewm(halflife=halflife, min_periods=min_periods).std().dropna()

In [5]:
from crypto_calculator.models import FactorReturns

return_data = FactorReturns.objects.values("market", "date")
df = pd.DataFrame(list(return_data))
df["date"] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df.set_index("date", inplace=True, drop=True)

OperationalError: no such table: factor_returns

In [6]:
symbol = "BTC-USD"

with sqlite3.connect(
    os.path.join(DATABASE_LOCATION, RETURN_DB)
) as conn:
    return_df = pd.read_sql_query(
        f"SELECT * FROM returns where symbol = '{symbol}'",
        conn,
    )


In [7]:
return_df

,id,date,return,risk_free_rate,total_return,symbol
0,0,2014-09-19,-0.069892,1.77,-0.069843,BTC-USD
1,1,2014-09-20,0.035686,1.77,0.035735,BTC-USD
2,2,2014-09-21,-0.024708,1.77,-0.024659,BTC-USD
3,3,2014-09-22,0.008303,1.77,0.008352,BTC-USD
4,4,2014-09-23,0.083598,1.77,0.083647,BTC-USD
...,...,...,...,...,...,...
3452,3452,2024-03-02,-0.006726,5.31,-0.006579,BTC-USD
3453,3453,2024-03-03,0.018191,5.31,0.018338,BTC-USD
3454,3454,2024-03-04,0.081588,5.31,0.081736,BTC-USD
3455,3455,2024-03-05,-0.066432,5.31,-0.066284,BTC-USD
